In [1]:
from dotenv import load_dotenv

load_dotenv()

True

이전 벡터스토어 예제에서 /chroma_db에 Chroma를 저장까지 했다는 가정하에 실습을 진행하도록 하겠습니다. <br>
벡터스토어는 먼저 로드해두겠습니다.

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma(persist_directory="./chroma_db", 
                     embedding_function=OpenAIEmbeddings())

# Retriever

랭체인에서 VectorStore 객체는 Runnable 객체가 아니라 LangChain Expression Language chains에 연결이 되지 않지만, Retriever는 Runnable 객체이므로 바로 LCEL 체인에 적용될 수 있습니다. <br>

## RunnableLambda를 사용한 구현

이제 RunnableLambda를 사용하여 간단한 리트리버를 구현해보겠습니다. <br>
벡터스토어 내의 유사도 분석 메서드를 사용했습니다.

In [3]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Shrimps are so delicious')]]

## 벡터스토어 as_retriever 메서드로 바로 구현

벡터스토어에는 as_retriever라는 메서드를 사용하여 간단하게 리트리버를 구현할 수 있습니다. <br>
이 때는 당연히 러너블람다로 객체를 감쌀 필요가 없습니다. 이미 러너블로 구현되어 있으니까요. <br>

벡터스토어는 다양한 retriever 서치 방법을 구현했는데요. 유사도 기반이나 MMR 등 다양한 방법이 있으며 벡터스토어마다 방식이 다를 수 있으니 사용하는 벡터스토어의 공식 문서에서 확인해보는것이 좋을 것 같습니다.

In [4]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Shrimps are so delicious')]]

## LCEL 연결해보기

이제 retriever를 만들어보았으니 체인에 연결해보는 작업을 진행해보겠습니다.

In [5]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=1000,
)

프롬프트를 설정하고 Retriever를 연결하여 질문에 대해 리트리버를 통해 Context를 생성한 후 LLM이 응답하는 형식으로 체인을 구현하였습니다.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

문서에서 데이터를 가져와 적절하게 대답하는 걸 확인할 수 있습니다.

In [7]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

Cats are independent pets that often enjoy their own space.


Retriever에 대해서는 다양한 예제가 많으므로 공식 문서를 잘 살펴보는 것이 좋을 것 같습니다. <br>
공식 문서 링크는 다음과 같습니다. <br>

https://python.langchain.com/v0.2/docs/how_to/#retrievers